In [1]:
%%writefile app.py
import pickle
import pandas as pd
import numpy as np
import streamlit as st
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

def main():
    model_SVM = load_model("SVM_model.sav")
    model_Bagging = load_model("BaggingClassifier.sav")
    model_Neural = load_model_tfkeras("TensorFlow Classification2")
    test_data = load_test_data("neo_task_new_final.csv")
    data1 = load_data("neo_task_new_final.csv")
    page = st.sidebar.selectbox(
        "Выберите страницу",
        ["Описание набора данных", "Визуализация зависимостей в наборе данных" ,"Получить предсказание моделей"]
    )

    if page == "Описание набора данных":
        st.title("Описание набора данных")
        st.write("Выберите страницу слева")

        st.header("Описание задачи")
        st.markdown("""Набор данных описывает ближайшие к Земле объекты, необходимо определить является ли объект опасным или нет""")

        st.write(data1.iloc[:10, :])

        st.header("Описание данных")
        st.markdown("""
Числовые признаки датасета:

* id - идентификатор объекта.
* est_diameter_min - минимальный диаметр объекта.
* est_diameter_max - максимальный диаметр объекта.
* relative_velocity - относительная скорость объекта.
* miss_distance - расстояние до объекта.
* absolute_magnitude - абсолютная звёздная величина объекта.

Бинапный признак датасета:

* hazardous - опасен объек(1) или нет(0).
""")

    elif page == "Получить предсказание моделей":
        st.title("Получить предсказание моделей")
        st.write("Выберите страницу слева")

        id = st.number_input("Задайте идентификатор объекта: ", 0.0, 5.427591e+07, 1.438528e+07)
        id = float(id)

        est_diameter_min = st.number_input("Задайте минимальный диаметр объекта: ", 0.0, 3.789265e+01, 0.127432)
        est_diameter_min = float(est_diameter_min)

        est_diameter_max = st.number_input("Задайте максимальный диаметр объекта:", 0.0, 8.473054e+01, 0.284911)
        est_diameter_max = float(est_diameter_max)

        relative_velocity  = st.number_input("Задайте относительную скорость объекта:", 0.0, 2.369901e+05, 48066.003701)
        relative_velocity  = float(relative_velocity)

        miss_distance  = st.number_input("Задайте расстояние до объекта:", 0.0, 7.479865e+07, 3.706655e+07)
        miss_distance  = float(miss_distance)

        absolute_magnitude  = st.number_input("Задайте абсолютную звёздную величину объекта:", 0.0, 3.320000e+01, 23.526653)
        absolute_magnitude  = float(absolute_magnitude)
        

        if st.button('Получить предсказание'):
            data = [id, est_diameter_min, est_diameter_max, relative_velocity, miss_distance, absolute_magnitude]
            data = np.array(data).reshape((1, -1))
            data_df = pd.DataFrame(data)
            pred1 = model_SVM.predict(data_df)
            st.write(f"Значение, предсказанное с помощью SVM: {pred1[0]:.2f}")
            pred2 = model_Bagging.predict(data_df)
            st.write(f"Значение, предсказанное с помощью GradientBoosting: {pred2[0]:.2f}")
            pred3 = np.around(model_Neural.predict(data_df, verbose=None))
            st.write(f"Значение, предсказанное с помощью полносвязной нейронной сети: {pred3[0][0]:.2f}")
        else:
            pass
    elif page == "Визуализация зависимостей в наборе данных":
        st.title("Визуализация зависимостей в наборе данных")
        st.write("Выберите страницу слева")
        request = st.selectbox("Выберите запрос",["Оценки плотности вероятности","Диаграмма корреляции", "Диаграмма рассеивания", "Круговая диаграмма"])
        if request == "Диаграмма корреляции":
            fig, ax = plt.subplots(figsize=(25,25), dpi= 60)
            sns.heatmap(data1.corr(), ax=ax, annot = True)
            st.write(fig)
        
        elif request == "Диаграмма рассеивания":
            x_axis = st.selectbox("Выберите столбец для x-axis", data1.columns)
            y_axis = st.selectbox("Выберите столбец для y-axis", data1.columns)
            visualize_data(data1, x_axis, y_axis)
        elif request == "Круговая диаграмма":
            labelss = st.selectbox("Выберите столбец", datacolumns("binary"))
            visual(data1, labelss)
        elif request == "Оценки плотности вероятности":
            x_title = st.selectbox("Выберите признак", datacolumns("numeric"))
            g = sns.displot(data=data1, x=x_title, hue='hazardous', log_scale=True, kde=True)
            chart = g.fig
            st.pyplot(chart)


def datacolumns(typee):
    if typee == "binary":
        return ["hazardous"]
    if typee == "numeric":
        return ["id", "est_diameter_min", "est_diameter_max", "relative_velocity", "miss_distance", "absolute_magnitude"]
def visual(data1, labelss):
    fig, ax = plt.subplots()
    data1.groupby(labelss).size().plot(kind='pie')
    plt.show()
    st.write(fig)
def visualize_data(data1, x_axis, y_axis):
    fig, ax = plt.subplots(figsize=(16,14)) 
    ax.scatter(x = data1[x_axis], y = data1[y_axis])
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.show()
    st.write(fig)
def load_model(path_to_file):
    model = pickle.load(open(path_to_file, 'rb'))
    #with open(path_to_file, 'rb') as model_file:
     #   model = pickle.load(model_file)
    return model

def load_model2(path_to_file):
    with open(path_to_file, 'rb') as model_file:
        model = joblib.load(model_file)
    return model

def load_model_tfkeras(path_to_file):
    model = tf.keras.models.load_model(path_to_file)
    #with open(path_to_file, 'rb') as model_file:
     #   model = tf.keras.models.load_model(model_file.name)
    return model
@st.cache_data
def load_test_data(path_to_file):
    df = pd.read_csv(path_to_file)
    df = df.drop(labels=['hazardous'], axis=1)
    return df
def load_data(path_to_file):
    df = pd.read_csv(path_to_file)
    return df
if __name__ == "__main__":
    main()
    

Overwriting app.py
